In [4]:
using Pkg
Pkg.add("Oceananigans")

using Oceananigans
using Oceananigans.Units

   Resolving package versions...
  No Changes to `C:\Users\Tom Cummings\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Tom Cummings\.julia\environments\v1.11\Manifest.toml`


In [5]:
Lx = 1000kilometers # east-west extent [m]
Ly = 1000kilometers # north-south extent [m]
Lz = 1kilometers    # depth [m]

grid = RectilinearGrid(size = (48, 48, 8),
                       x = (0, Lx),
                       y = (-Ly/2, Ly/2),
                       z = (-Lz, 0),
                       topology = (Periodic, Bounded, Bounded))

48×48×8 RectilinearGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo
├── Periodic x ∈ [0.0, 1.0e6)          regularly spaced with Δx=20833.3
├── Bounded  y ∈ [-500000.0, 500000.0] regularly spaced with Δy=20833.3
└── Bounded  z ∈ [-1000.0, 0.0]        regularly spaced with Δz=125.0

In [8]:



α = 0

# Background deformation flow
Ū(x, y, z, t) = -α * x
V̄(x, y, z, t) = α*y  # No y-dependence in 2D

model = NonhydrostaticModel(
    grid = grid,
    coriolis = BetaPlane(latitude = -45),
    buoyancy = BuoyancyTracer(),
    tracers = :b,
    background_fields = (u=Ū, v=V̄),  # Adds deformation flow
    advection = WENO(),              # High-order advection for fronts
    #closure = ScalarDiffusivity(ν=1e-5, κ=1e-5),
    )

NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 48×48×8 RectilinearGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO(order=5)
├── tracers: b
├── closure: Nothing
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: BetaPlane{Float64}(f₀=-0.000103126, β=1.61868e-11)

In [ ]:
"""
    ramp(y, Δy)

Linear ramp from 0 to 1 between -Δy/2 and +Δy/2.

For example:
```
            y < -Δy/2 => ramp = 0
    -Δy/2 < y < -Δy/2 => ramp = y / Δy
            y >  Δy/2 => ramp = 1
```
"""
ramp(y, Δy) = min(max(0, y/Δy + 1/2), 1)

N² = 1e-5 # [s⁻²] buoyancy frequency / stratification
M² = 1e-7 # [s⁻²] horizontal buoyancy gradient

Δy = 100kilometers # width of the region of the front
Δb = Δy * M²       # buoyancy jump associated with the front
ϵb = 1e-2 * Δb     # noise amplitude

bᵢ(x, y, z) = N² * z + Δb * ramp(y, Δy) + ϵb * randn()

set!(model, b=bᵢ)

In [ ]:
# Forcing terms (if not using background_fields)
u_forcing(x, y, z, t, u, v, w, b) = α * u + coriolis.f₀ * v
v_forcing(x, y, z, t, u, v, w, b) = -α * v - coriolis.f₀ * u